In [1]:
import pandas as pd
import os 
import datetime as dt
import numpy as np

## Cass

In [2]:
Columns = pd.read_csv(r'Daily_Column_Dictionary.csv')

no_bound = (-np.inf,np.inf)
bounds = dict.fromkeys(Columns.columns[4:], no_bound)

def mask_outside(data, bounds):
    lower, upper = bounds
    data = data.mask(data< lower)
    data = data.mask(data> upper)
    return data

bounds['Li190'] = (-10, 2700)
bounds['Li200'] = (-10, 1300)
bounds['Air_Temp'] = (-40, 45)
bounds['5m_Temp'] = (-40, 45)
bounds['12m_Temp'] = (-40, 45)
bounds['Ground_Temp'] = (-40, 70)
bounds['Rel_Humidity'] = (0, 110)
bounds['Soil_Temp'] = (-40, 45)
bounds['10cm_Soil_Temp'] = (-40, 45)
bounds['20cm_Soil_Temp'] = (-40, 45)
bounds['50cm_Soil_Temp'] = (-40, 45)
bounds['Wind_Speed'] = (0,150)

bounds['Li190_Min'] = (-10, 2700)
bounds['Li200_Min'] = (-10, 2700)
bounds['Air_Temp_Min'] = (-40, 45)
bounds['5m_Temp_Min'] = (-40, 45)
bounds['12m_Temp_Min'] = (-40, 45)
bounds['Ground_Temp_Min'] = (-40, 70)
bounds['Rel_Humidity_Min'] = (0, 110)
bounds['Soil_Temp_Min'] = (-40, 45)
bounds['10cm_Soil_Temp_Min'] = (-40, 45)
bounds['20cm_Soil_Temp_Min'] = (-40, 45)
bounds['50cm_Soil_Temp_Min'] = (-40, 45)
bounds['Wind_Speed_Min'] = (0,150)

bounds['Li190_Max'] = (-10, 2700)
bounds['Li200_Max'] = (-10, 2700)
bounds['Air_Temp_Max'] = (-40, 45)
bounds['5m_Temp_Max'] = (-40, 45)
bounds['12m_Temp_Max'] = (-40, 45)
bounds['Ground_Temp_Max'] = (-40, 70)
bounds['Rel_Humidity_Max'] = (0, 110)
bounds['Soil_Temp_Max'] = (-40, 45)
bounds['10cm_Soil_Temp_Max'] = (-40, 45)
bounds['20cm_Soil_Temp_Max'] = (-40, 45)
bounds['50cm_Soil_Temp_Max'] = (-40, 45)
bounds['Wind_Speed_Max'] = (0,150)

bounds['Wind_Dir'] = (0,360)
bounds['Wind_Dir_STD'] = (0,np.inf)
bounds['Rain'] = (0,1825)

## Cass

In [3]:
Raw_Dir = r'Raw_Data/Cass_Daily/' # Input 

Parsed_Dir = r'Formatted_Data/Cass_Daily/' # Output
if not os.path.exists(Parsed_Dir):
    os.makedirs(Parsed_Dir)
    
Filtered_Dir = r'Filtered_Data/Cass_Daily/' # Output
if not os.path.exists(Filtered_Dir):
    os.makedirs(Filtered_Dir)
    
ymin = 1997 # Start of series
ymax = 2020 # End of series
years = range(ymin,ymax+1) 

In [37]:
for y in years:
    print(y)
    data = pd.read_csv(Raw_Dir+'/Cass_'+str(y)+'_Daily.csv')
    data = data[~data.index.duplicated(keep='first')]
    out_data = pd.DataFrame()
    filtered_data = pd.DataFrame()

    data.index = pd.to_datetime(data.Year,format='%Y')+pd.to_timedelta(data.Day-1,unit='D')
    out_data.index = pd.date_range(dt.datetime(y,1,1),dt.datetime(y,12,31))
    filtered_data.index = pd.date_range(dt.datetime(y,1,1),dt.datetime(y,12,31))

    # Attach data to out_data
    for C in Columns.columns[4:]:
        for c in Columns[C]:
            if c in list(data.columns):
                try:
                    # See if value can be interpreted as numeric
                    out_data[C] = pd.to_numeric(data[c], errors='coerce')
                    break
                except:
                    pass
    
    # Filter and attach data to filtered_data
    for C in Columns.columns[4:]:
        for c in Columns[C]:
            if c in list(data.columns):
                try:
                    # See if value can be interpreted as numeric
                    filtered_data[C] = mask_outside(pd.to_numeric(data[c], errors='coerce'), bounds[C])
                    break
                except:
                    pass
        
    out_data.index.names = ['Time'] # Set Index name to Time
    out_data.to_csv(Parsed_Dir+'Cass_'+str(y)+'_Daily.csv') # Save to file
    
    filtered_data.index.names = ['Time']
    filtered_data.to_csv(Filtered_Dir+'/Cass_'+str(y)+'_Daily.csv')
    

1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017


C:\Users\callu\AppData\Local\Temp\ipykernel_20164\2377194124.py:18: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  out_data[C] = pd.to_numeric(data[c], errors='coerce')
C:\Users\callu\AppData\Local\Temp\ipykernel_20164\2377194124.py:29: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  filtered_data[C] = mask_outside(pd.to_numeric(data[c], errors='coerce'), bounds[C])


2018
2019
2020


## Reformat Chilton

In [43]:
Raw_Dir = r'Raw_Data/Chilton_Daily/' # Input 
Parsed_Dir = 'Formatted_Data/Chilton_Daily/' # Output
if not os.path.exists(Parsed_Dir):
    os.makedirs(Parsed_Dir)

Filtered_Dir = 'Filtered_Data/Chilton_Daily/' # Output
if not os.path.exists(Filtered_Dir):
    os.makedirs(Filtered_Dir)

files  = [f for f in os.listdir(Raw_Dir) if f.endswith('.csv')] # Find datafiles 
files.reverse()


In [42]:
for file in files:
    print('Reading '+file+'...')
    data = pd.read_csv(Raw_Dir+file)
    data.index = pd.to_datetime(data['date'],format='%m/%d/%Y')

    # Find timespan of dataset
    ymin = min(data.index).year
    ymax = max(data.index).year
    years = range(ymin,ymax+1) 
    data = data[~data.index.duplicated(keep='first')] # Drop any duplicate rows
    

    for y in years:
        print(y)
        # Create container for data
        out_data = pd.DataFrame()
        out_data.index = pd.date_range(dt.datetime(year=y,month=1,day=1),dt.datetime(year=y+1,month=1,day=1),freq='D')

        # Create container for filtered data
        filtered_data = pd.DataFrame()
        filtered_data.index = pd.date_range(dt.datetime(year=y,month=1,day=1),dt.datetime(year=y+1,month=1,day=1),freq='D')
    
        # Attach data to out_data
        for C in Columns.columns[4:]:
            for c in Columns[C]:
                if c in list(data.columns):
                    try:
                        # See if value can be interpreted as numeric
                        out_data[C] = pd.to_numeric(data[c], errors='coerce')
                        break
                    except:
                        pass
                    
        # Filter and attach data to filtered_data
        for C in Columns.columns[4:]:
            for c in Columns[C]:
                if c in list(data.columns):
                    try:
                        # See if value can be interpreted as numeric
                        filtered_data[C] = mask_outside(pd.to_numeric(data[c], errors='coerce'), bounds[C])
                        break
                    except:
                        pass

        out_data.index.names = ['Time'] # Set Index name to Time
        out_data.to_csv(Parsed_Dir+'Chilton_'+str(y)+'_Daily.csv') # Save to file
        
        filtered_data.index.names = ['Time'] # Set Index name to Time
        filtered_data.to_csv(Filtered_Dir+'Chilton_'+str(y)+'_Daily.csv') # Save to file

Reading Chilton_Daily_86_00.csv...
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
Reading Chilton_Daily_01_06.csv...
2001
2002
2003
2004
2005
2006


In [ ]:
mask_outside(pd.to_numeric(out_data.Air_Temp_Min, errors='coerce'),(0,19))

Time
2006-01-01   NaN
2006-01-02   NaN
2006-01-03   NaN
2006-01-04   NaN
2006-01-05   NaN
              ..
2006-12-28   NaN
2006-12-29   NaN
2006-12-30   NaN
2006-12-31   NaN
2007-01-01   NaN
Freq: D, Name: Air_Temp_Min, Length: 366, dtype: float64

## Sync daily with cleaned hourly readings

In [70]:
def cleanHourly_dirtyDaily(hourly_path, daily_path, station_name, out_path, years):
    
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    
    for y in years:
        print(y,end='\r')
        cleaned_data = pd.read_csv(hourly_path+station_name+'_'+str(y)+'_Hourly.csv')
        data = pd.read_csv(daily_path+station_name+'_'+str(y)+'_Daily.csv')

        data.Time = pd.to_datetime(data.Time,format='%Y-%m-%d')
        cleaned_data.Time = pd.to_datetime(cleaned_data.Time)
        out_data = data.copy()

        for i, day in enumerate(data.Time):
            hours = pd.date_range(day+dt.timedelta(hours=-23), day+dt.timedelta(hours=0),freq='H').tolist()
            day_data = cleaned_data[cleaned_data.Time.isin(hours)]
            for var in cleaned_data.columns.tolist():
                daily_vars = [c for c in data.columns if c.startswith(var)]
                if any(day_data[var].isna()):
                    for daily_var in daily_vars:
                        out_data.loc[i,daily_var]=np.nan


        out_data.to_csv(out_path+'/'+station_name+'_'+str(y)+'_Daily.csv',index=False)
        

In [71]:
hourly_path = 'Cleaned_Data/Chilton_Hourly/'
daily_path = 'Filtered_Data/Chilton_Daily/'
station_name = 'Chilton'
out_path = 'Cleaned_Data/Chilton_Daily/'
years = range(1986,2007)
cleanHourly_dirtyDaily(hourly_path, daily_path, station_name, out_path, years)

In [47]:
hourly_path = 'Cleaned_Data/Cass_Hourly/'
daily_path = 'Filtered_Data/Cass_Daily/'
station_name = 'Cass'
out_path = 'Cleaned_Data/Cass_Daily/'
years = range(1997,2020)
cleanHourly_dirtyDaily(hourly_path, daily_path, station_name, out_path, years)

## Parse and filter ECAN rain gauge data

In [48]:
def parse_ECAN_rain(datapath, Parsed_Dir, Filtered_Dir):
    data  = pd.read_csv(datapath)
    data.Time = pd.to_datetime(data.Time,format='%d/%m/%Y %H:%M')
    years = range(min(pd.to_datetime(data.Time,format='%d/%m/%Y %H:%M')).year,
                  max(pd.to_datetime(data.Time,format='%d/%m/%Y %H:%M')).year+1)
    for y in years:
        print(y, end='\r')
        
        out_data = pd.DataFrame()
        out_data.index = pd.date_range(dt.datetime(y,1,1),dt.datetime(y,12,31))
        data.index = data.Time
        data.index.names = ['Time']

        out_data = pd.DataFrame()
        out_data.index = pd.date_range(dt.datetime(year=y,month=1,day=1),dt.datetime(year=y+1,month=1,day=1),freq='D')

        # Create container for filtered data
        filtered_data = pd.DataFrame()
        filtered_data.index = pd.date_range(dt.datetime(year=y,month=1,day=1),dt.datetime(year=y+1,month=1,day=1),freq='D')
    
        # Attach data to out_data
        for C in Columns.columns[4:]:
            for c in Columns[C]:
                if c in list(data.columns):
                    try:
                        # See if value can be interpreted as numeric
                        out_data[C] = pd.to_numeric(data[c], errors='coerce')
                        break
                    except:
                        pass
                    
        # Filter and attach data to filtered_data
        for C in Columns.columns[4:]:
            for c in Columns[C]:
                if c in list(data.columns):
                    try:
                        # See if value can be interpreted as numeric
                        filtered_data[C] = mask_outside(pd.to_numeric(data[c], errors='coerce'), bounds[C])
                        break
                    except:
                        pass

        out_data.index.names = ['Time'] # Set Index name to Time
        out_data.to_csv(Parsed_Dir+'_'+str(y)+'_Daily.csv') # Save to file
        
        filtered_data.index.names = ['Time'] # Set Index name to Time
        filtered_data.to_csv(Filtered_Dir+'_'+str(y)+'_Daily.csv') # Save to file
 

In [54]:
Stations = ['ArthursRain','CarringtonRain','GrasmereRain','EskRain']
for f in Stations:
    datapath = 'Raw_Data/'+f+'/'+f+'.csv'
    Parsed_Dir = 'Formatted_Data/'+f+'_Daily/'+f
    Filtered_Dir = 'Filtered_Data/'+f+'_Daily/'+f
    
    if not os.path.exists('Formatted_Data/'+f+'_Daily/'):
        os.makedirs('Formatted_Data/'+f+'_Daily/')
    if not os.path.exists('Filtered_Data/'+f+'_Daily/'):
        os.makedirs('Filtered_Data/'+f+'_Daily/')
    
    parse_ECAN_rain(datapath, Parsed_Dir, Filtered_Dir)

## Format Cliflo Datasets
Cliflo datasets are all in a universal format, as such there is a function called Parse_Cliflo in the Database_Utils.py library for you to use. 

In [7]:
Stations

['ArthursCombined',
 'ArthursEWS',
 'ArthursStoreRain',
 'ArthursTemps',
 'ArthursWind',
 'BealeyRain',
 'BrokenRiverCombined',
 'BrokenRiverRain',
 'BrokenRiverTemp',
 'BrokenRiverWind',
 'CampStreamCombined',
 'CampStreamRain',
 'CampStreamWind',
 'CastleHillRain',
 'ClifloArthursRain',
 'CragieburnForestCombined',
 'CragieburnForestTemps',
 'CragieburnForestWind',
 'CragieburnStnRain',
 'FlockhillRain',
 'HarperRiverTemps',
 'MtPhilistineEWSCombined',
 'MtPhilistineTemps',
 'MtWhiteRain',
 'OldCassRain',
 'OtiraTemps']

In [6]:
# Get a list of all the stations in /Raw_Data/Cliflo_Daily
Stations = [f[:-4][:-6] for f in os.listdir('Raw_Data/Cliflo_Daily/') if f.endswith('txt')]

import Database_Utils as DU

for f in Stations:
    datapath = 'Raw_Data/Cliflo_Daily/'+f+'_Daily.txt'
    Parsed_Dir = 'Formatted_Data/'+f+'_Daily/'+f
    Filtered_Dir = 'Filtered_Data/'+f+'_Daily/'+f
    
    if not os.path.exists('Formatted_Data/'+f+'_Daily/'):
        os.makedirs('Formatted_Data/'+f+'_Daily/')
    if not os.path.exists('Filtered_Data/'+f+'_Daily/'):
        os.makedirs('Filtered_Data/'+f+'_Daily/')
    
    DU.Parse_Cliflo(datapath, Parsed_Dir, Filtered_Dir, Columns, bounds)

In [5]:
f

'CragieburnForestRain'